set [SDC](https://commons.wikimedia.org/wiki/Commons:Structured_data#:~:text=Structured%20data%20on%20Commons%20is,from%20Wikidata%2C%20Wikimedia's%20knowledge%20base.) depicts from files uploaded in [spa2Commons](https://commons.wikimedia.org/wiki/Category:Uploaded_with_spa2Commons)

### Theory depicts
* all pictures from [SPA](https://portrattarkiv.se/about) depicts one person
* one category in WIkicommons is connected to the person that the picture depicts

#### How to 
##### Using Minefield and a pile 
see below _1. get WD object the Picture depict_
* a csv file is created from a pile using
  * petscan creates the pile ex. [set output pagepile](https://petscan.wmflabs.org/?psid=20485153)--> [pagepile 39223](https://pagepile.toolforge.org/api.php?action=get_data&id=39223)
     * example [petscan files modified after 20211103](https://petscan.wmflabs.org/?psid=20577669)
  * [minefield](https://hay.toolforge.org/minefield/) creates the csv file  
    * see [gist](https://gist.github.com/salgo60/b5d05fae5c865b678edb338b09e4b302)
* [video](https://www.youtube.com/watch?v=FUoG0veIeMY&feature=youtu.be)     
    
### Theory SPA identifier
* a picture uploaded to Wikicommons using [SPA2common javascript](https://github.com/salgo60/spa2Commons) will have the link to the SPA picture in the template Information and param **Source** ex. [File:Axel_Sammuli_SPA3.jpg](https://commons.wikimedia.org/wiki/File:Axel_Sammuli_SPA3.jpg) -->  SPA id = [SPA idYB0QHyfj0hAAAAAAAAAf8g](https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAAAf8g)
#### How to
* read all pictures in category  [spa2Commons](https://commons.wikimedia.org/wiki/Category:Uploaded_with_spa2Commons) and check param **Source**
  * this is done in a [PAWS notebook](https://hub.paws.wmcloud.org/user/Salgo60/notebooks/Traverse%20category%20to%20find%20SPA%20id%20.ipynb)
### Misc
* [SPARQL mwapi](https://en.wikibooks.org/wiki/SPARQL/SERVICE_-_mwapi)
* [API:Categories](https://www.mediawiki.org/wiki/API:Categories)
* Test SPARQL
   * [get files in Categories](https://wcqs-beta.wmflabs.org/#%23Wikidata%20items%20of%20files%20in%20Category%3AArtworks%20with%20structured%20data%20with%20redirected%20P6243%20property%0ASELECT%20%3Ffile%20%3Ftitle%20%20%3Fspa%20%0AWITH%0A%7B%0A%20%20SELECT%20%3Ffile%20%3Ftitle%0A%20%20WHERE%0A%20%20%7B%0A%20%20%20%20SERVICE%20wikibase%3Amwapi%0A%20%20%20%20%7B%0A%20%20%20%20%20%20bd%3AserviceParam%20wikibase%3Aapi%20%22Generator%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20wikibase%3Aendpoint%20%22commons.wikimedia.org%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20mwapi%3Agcmtitle%20%22Category%3AUploaded_with_spa2Commons%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20mwapi%3Agenerator%20%22categorymembers%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20mwapi%3Agcmtype%20%22file%22%20.%0A%20%20%20%20%20%20bd%3AserviceParam%20mwapi%3Agcmlimit%20%22max%22%20.%0A%20%20%20%20%20%20%3Ftitle%20wikibase%3AapiOutput%20mwapi%3Atitle%20.%0A%20%20%20%20%20%20%3Fpageid%20wikibase%3AapiOutput%20%22%40pageid%22%20.%0A%20%20%20%20%7D%0A%20%20%20%20BIND%20%28URI%28CONCAT%28%27https%3A%2F%2Fcommons.wikimedia.org%2Fentity%2FM%27%2C%20%3Fpageid%29%29%20AS%20%3Ffile%29%0A%20%20%7D%0A%7D%20AS%20%25get_files%0AWHERE%0A%7B%0A%20%20INCLUDE%20%25get_files%0A%20%20OPTIONAL%20%7B%3Ffile%20wdt%3AP4819%20%3Fspa%7D%0A%7D)
* [PAWS Example Notebooks](https://wikitech.wikimedia.org/wiki/PAWS/PAWS_examples_and_recipes)   
* [hub.toolforge.org](https://hub.toolforge.org)
* [writeSDoCfromExcel](https://github.com/KBNLwikimedia/SDoC/blob/main/writeSDoCfromExcel/WriteSDoCfromExcel_nopasswd.py)

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-11-26 02:50:01.339986


In [2]:
import requests

S = requests.Session()

URL = "https://commons.wikipedia.org/w/api.php"

def get_Category(pageName):
    PARAMS = {
        "action": "query",
        "format": "json",
        "prop": "categories",
        "titles": pageName   
    }
    #print (PARAMS)
    r = S.get(url=URL, params=PARAMS)
    data = r.json()
    # TODO dont get hidden categories
    filtercat = {
        "Category:CC-BY-4.0",
        "Category:Swedish Portrait Archive",
        "Category:Uploaded with spa2Commons",
        "Category:Template Unknown (author)",
        "Category:Images with extracted images",
        "Category:Extracted images",
        "Category:Scanned with HP Deskjet F4200",
        "Category:Pages using duplicate arguments in template calls",
        "Category:Creative Commons Attribution-Share Alike missing SDC copyright status",
        "Category:Creative Commons Attribution-Share Alike 4.0 missing SDC copyright license",
        "Category:Creative Commons Attribution missing SDC copyright status",
        "Category:Creative Commons Attribution 4.0 missing SDC copyright license",
        "Category:Media requiring renaming - rationale 3",
        "Media requiring renaming - target already exists"
        }
    target_category = ""
    PAGES = data["query"]["pages"]
    for k, v in PAGES.items():
    #    print(k,v)
        for cat in v['categories']:
            if cat["title"] not in filtercat:
                target_category = cat["title"]
                #print("\tTarget cat" ,target_category)
    return target_category

In [3]:
common_name = "File:A_G_Ahlqvist_SPA10.jpg"
get_Category(common_name)

'Category:Alfred Gustaf Ahlqvist'

In [4]:
def getWD(commonsCategory):
    urlHub = "https://hub.toolforge.org/commons:" + commonsCategory + "?format=json&site=wd"
    #print(urlHub)
    hub = S.get(url=urlHub)
    data = hub.json()
    try:
        wd = data["destination"]["url"].replace("https://www.wikidata.org/wiki/","")    
    except:
        print("Error", data)
        wd =""
    return wd

In [5]:
test_category ="Category:Axel_Rappe_(1838%E2%80%931918)"
getWD(test_category)

'Q4830349'

### 1. get WD object the Picture depict

In [6]:
# used tool xxxx to get csv files with pictures 
import csv

mid_wd_list = []
cat2commonsfiles = "Cat_2commons.csv"
cat2commonsfiles = "Cat_2commons_20211101.csv"
cat2commonsfiles = "Cat_2commons_20211103.csv"
cat2commonsfiles = "Cat_2commons_20211106.csv"
cat2commonsfiles = "Cat_2commons_20211114.csv"
cat2commonsfiles = "Cat_2commons_20211126.csv"


with open(cat2commonsfiles) as csvfile:
    cat_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
#    cat_reader = csv.DictReader(csvfile, delimiter=';', quotechar='"')
#    cat_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in cat_reader:
#        print(row)
#        print(row["mid"],get_Category(row["title"]),row["url"]) 
#        print(row["mid"],getWD(get_Category(row["title"]))) 
        mid_wd_list.append([row["mid"],getWD(get_Category(row["title"]))])
        #print(mid_wd_list)
        


Error {'message': 'Not Found', 'context': {'text': 'commons:Category:Bernhard Lundgren 1843', 'lang': 'en'}}
Error {'message': 'Not Found', 'context': {'text': 'commons:Category:Bernhard Lundgren 1843', 'lang': 'en'}}
Error {'message': 'Not Found', 'context': {'text': 'commons:Category:Axel Wästfelt 1881', 'lang': 'en'}}
Error {'message': 'Not Found', 'context': {'text': 'commons:Category:Axel Wästfelt 1881', 'lang': 'en'}}
Error {'message': 'Not Found', 'context': {'text': 'commons:Category:Karl Salin 1890', 'lang': 'en'}}
Error {'message': 'Not Found', 'context': {'text': 'commons:Category:Karl Salin 1890', 'lang': 'en'}}
Error {'message': 'Not Found', 'context': {'text': 'commons:Category:Bernhard Lundgren 1843', 'lang': 'en'}}
Error {'message': 'Not Found', 'context': {'text': 'commons:Category:Sven Trägårdh 1814', 'lang': 'en'}}
Error {'message': 'Not Found', 'context': {'text': 'commons:Category:Sven Trägårdh 1814', 'lang': 'en'}}


In [7]:
import pandas as pd
df = pd.DataFrame(mid_wd_list)
df.head(10)

,0,1
0,M110465275,Q2824703
1,M111371972,Q2482314
2,M112375203,Q18238225
3,M112375216,Q18238225
4,M112392665,Q4943098
5,M112408085,Q29047398
6,M112427640,Q48709014
7,M112458663,Q6171709
8,M112489160,Q5771716
9,M112490909,Q109545314


In [8]:
df.to_csv("SPACategories_Mid_WD.txt")
df.shape

(1717, 2)

In [9]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2021-11-26 03:03:52.587248
Time elapsed (hh:mm:ss.ms) 0:13:51.248001
